## Importamos librerias

In [54]:
import numpy as np
import os
import librosa
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from sklearn.calibration import LabelEncoder
from keras.layers import LSTM, Dense, Dropout, Conv1D

## Preparativos antes de la RNN

In [2]:
def preprocess_audio(file_path,max_len):
    # Cargar el audio
    signal, sr = librosa.load(file_path,sr=96000)
    # Realizar preénfasis
    #filter_audio = librosa.effects.preemphasis(signal)
    # Extraer MFCCs
    mfcc = librosa.feature.mfcc(y=signal, sr=sr, n_mfcc=13)

    # Realizar padding o recorte
    if mfcc.shape[1] < max_len:
        num_zeros = max_len - mfcc.shape[1]
        padded_mfcc = np.pad(mfcc, ((0, 0), (0, num_zeros)), mode='constant', constant_values=0)
        return padded_mfcc
    else:
        mfcc = mfcc[:, :max_len]
        return mfcc

In [3]:
DATA_DIR = 'Data'
LABELS = ['happy','cat', 'bed']

mfccs = []
labels = []

padding = 500

for label in LABELS:
    path_file = DATA_DIR + f'/{label}'
    for file in os.listdir(path_file):
        file_path = path_file + f'/{file}'
        
        mfcc = preprocess_audio(file_path,padding)
        
        mfccs.append(mfcc)
        labels.append(label)

In [4]:
#Convertimos las listas a numpy array
mfccs_array = np.array(mfccs) #Matriz 3d (5187, 13, 500)

In [5]:
# Convertir etiquetas a números
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

print("Codificación realizada:")
for idx, label in enumerate(le.classes_):
    print(f"{label} -> {idx}")

Codificación realizada:
bed -> 0
cat -> 1
happy -> 2


In [59]:
# Dividir datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(mfccs_array, labels_encoded, test_size=0.2, random_state=312)

# Crear modelo LSTM (Long short-term memory)
model = Sequential([
    Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(128, return_sequences=True),
    Dropout(0.3),
    LSTM(128),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(3, activation='softmax')  # 3 clases: 'happy', 'cat', 'bed'
])

# Compilar el modelo
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Entrenamiento

In [60]:
# Entrenar el modelo
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=15, batch_size=32)

Epoch 1/15
130/130 [==============================] - 4s 16ms/step - loss: 0.8493 - accuracy: 0.6030 - val_loss: 0.6172 - val_accuracy: 0.7678
Epoch 2/15
130/130 [==============================] - 2s 12ms/step - loss: 0.6706 - accuracy: 0.7137 - val_loss: 0.5543 - val_accuracy: 0.7688
Epoch 3/15
130/130 [==============================] - 2s 13ms/step - loss: 0.5625 - accuracy: 0.7689 - val_loss: 0.5723 - val_accuracy: 0.7861
Epoch 4/15
130/130 [==============================] - 2s 12ms/step - loss: 0.5349 - accuracy: 0.7884 - val_loss: 0.4782 - val_accuracy: 0.8054
Epoch 5/15
130/130 [==============================] - 2s 12ms/step - loss: 0.4827 - accuracy: 0.8089 - val_loss: 0.4546 - val_accuracy: 0.8304
Epoch 6/15
130/130 [==============================] - 2s 12ms/step - loss: 0.4506 - accuracy: 0.8195 - val_loss: 0.5277 - val_accuracy: 0.8131
Epoch 7/15
130/130 [==============================] - 2s 13ms/step - loss: 0.4258 - accuracy: 0.8313 - val_loss: 0.4654 - val_accuracy: 0.8170

## Evaluación

In [61]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

33/33 [==============================] - 0s 4ms/step - loss: 0.3183 - accuracy: 0.8863
Loss: 0.318325012922287
Accuracy: 0.8863198161125183


## Prueba

In [65]:
file_paths = ['audio_bed.wav','audio_cat.wav','audio_happy.wav']

for file_path in file_paths:
    print(file_path)
    preprocessed_audio = preprocess_audio(file_path,padding)
    preprocessed_audio = preprocessed_audio.reshape(1, preprocessed_audio.shape[0], preprocessed_audio.shape[1])  # Convertir a formato (1, features, tiempo)
    prediction = model.predict(preprocessed_audio)

    print(f"{le.inverse_transform([0])[0]} : {prediction[0][0] * 100}%")
    print(f"{le.inverse_transform([1])[0]} : {prediction[0][1] * 100}%")
    print(f"{le.inverse_transform([2])[0]} : {prediction[0][2] * 100}%")

    predicted_label_encoded = np.argmax(prediction, axis=1)[0]
    predicted_label = le.inverse_transform([predicted_label_encoded])[0]
    print(f"\nLa palabra predicha es: {predicted_label}\n")

audio_bed.wav
1/1 [==============================] - 0s 14ms/step
bed : 61.15589737892151%
cat : 32.22641050815582%
happy : 6.617692857980728%

La palabra predicha es: bed

audio_cat.wav
1/1 [==============================] - 0s 14ms/step
bed : 19.175419211387634%
cat : 78.44647765159607%
happy : 2.3781079798936844%

La palabra predicha es: cat

audio_happy.wav
1/1 [==============================] - 0s 14ms/step
bed : 1.6778495162725449%
cat : 96.5961754322052%
happy : 1.725967787206173%

La palabra predicha es: cat



## Guardamos el modelo

In [63]:
# Entrenar el modelo
model.save('GUI/modelo.h5')